In [69]:
import pandas as pd
import numpy as np

In [4]:
def calculate_cpr(term_months, prepayment_multiplier=1.0, seasoning=0, psa_1=0.002, psa_30=0.06):
    """
    Calculate the monthly Conditional Prepayment Rate (CPR) for a loan.

    Args:
        term_months (int): Term of the loan in months.
        prepayment_multiplier (float): Multiplier for prepayment (default 1.0, i.e., 100%).
        seasoning (int): Number of months seasoned (0-12).
        psa_1 (float): PSA CPR at month 1 (default 0.2% = 0.002).
        psa_30 (float): PSA CPR at month 30 (default 6% = 0.06).

    Returns:
        list: Monthly CPRs for each month from 1 to term_months.
    """
    psa_1 = psa_1*prepayment_multiplier
    psa_30 = psa_30*prepayment_multiplier
    cpr_list = []
    cpr_increment = psa_1 # Linear growth from month 1 to 30

    # Adjust starting CPR for seasoning
    if seasoning == 0:
        starting_cpr = psa_1
    else:
        starting_cpr = psa_1*seasoning

    for t in range(2, term_months + 2):
        if t <= 30-seasoning:
            cpr = starting_cpr + (t - 1) * cpr_increment
            cpr = min(cpr, psa_30)
        else:
            cpr = psa_30
        cpr_list.append(cpr)

    return cpr_list

In [5]:
def calculate_initial_monthly_payment(balance, annual_rate, term_months, seasoning_months=0):
    """
    Calculate the initial monthly payment for a mortgage.

    Args:
        balance (float): Remaining mortgage balance.
        annual_rate (float): Annual interest rate (e.g., 0.05 for 5%).
        term_months (int): Original term of the loan in months.
        seasoning_months (int): Number of months seasoned (payments already made).

    Returns:
        float: Initial monthly payment at the given seasoning.
    """
    # Remaining term after seasoning
    remaining_term = term_months - seasoning_months
    if remaining_term <= 0:
        raise ValueError("Seasoning months must be less than term of loan.")

    monthly_rate = annual_rate / 12
    if monthly_rate == 0:
        return balance / remaining_term

    payment = balance * (monthly_rate * (1 + monthly_rate) ** remaining_term) / ((1 + monthly_rate) ** remaining_term - 1)
    return payment

In [65]:
import pandas as pd

def cpr_smm_dataframe(balance, mortgage_rate, passthrough_rate, term_months,  interest_rate, prepayment_multiplier=1.0, seasoning=0, psa_1=0.002, psa_30=0.06):
    """
    Generate a DataFrame with CPR and SMM values for each month.

    Args:
        balance (float): Remaining mortgage balance.
        mortgage_rate (float): Annual mortgage rate (e.g., 0.05 for 5%).
        term_months (int): Term of the loan in months.
        prepayment_multiplier (float): Multiplier for prepayment (default 1.0).
        seasoning (int): Number of months seasoned (0-12).
        psa_1 (float): PSA CPR at month 1.
        psa_30 (float): PSA CPR at month 30.

    Returns:
        pd.DataFrame: DataFrame with columns 'CPR' and 'SMM'.
    """
    # Use the existing cpr_list variable, which is already calculated

    cpr_list = calculate_cpr(term_months, prepayment_multiplier, seasoning, psa_1, psa_30)
    df = pd.DataFrame({
        'CPR': cpr_list,
        'SMM': [1 - (1 - cpr) ** (1 / 12) for cpr in cpr_list]
        }, index=range(1, len(cpr_list) + 1))
    df['Start Month. Bal.'] = None
    df.at[1, 'Start Month. Bal.'] = balance
    df['Month. Paym.'] = None
    df.at[1, 'Month. Paym.'] = calculate_initial_monthly_payment(balance, mortgage_rate, term_months, seasoning)
    df["Int by Mort Hold"] = None
    df.at[1, "Int by Mort Hold"] = df.at[1, 'Start Month. Bal.'] * (mortgage_rate / 12)
    df["Int to P-T Inv"] = None
    df.at[1, "Int to P-T Inv"] = df.at[1, 'Start Month. Bal.'] * (passthrough_rate / 12)
    df["Scheduled Princ."] = None
    df.at[1, "Scheduled Princ."] = df.at[1, 'Month. Paym.'] - df.at[1, "Int by Mort Hold"]
    df["Prepaym."] = None
    df.at[1, "Prepaym."] = (df.at[1, "Start Month. Bal."]- df.at[1, "Scheduled Princ."])*(1-(1-df.at[1, 'CPR'])**(1/12))
    df["Total Princ."] = None
    df.at[1, "Total Princ."] = df.at[1, "Scheduled Princ."] + df.at[1, "Prepaym."]
    df["End Month. Bal."] = None
    df.at[1, "End Month. Bal."] = df.at[1, 'Start Month. Bal.'] - df.at[1, "Total Princ."]

    for i in range(2, len(df) + 1):
        if df.at[i-1, "End Month. Bal."] is not None and df.at[i-1, "End Month. Bal."] > 0:
            df.at[i, "Start Month. Bal."] = df.at[i-1, "End Month. Bal."]
            df.at[i, "Month. Paym."] = df.at[i-1, "End Month. Bal."] * (mortgage_rate / 12) / (1-(1+ mortgage_rate/12)**(-(term_months-seasoning-i+1)))
            df.at[i, "Int by Mort Hold"] = mortgage_rate* df.at[i-1, "End Month. Bal."] / 12
            df.at[i, "Int to P-T Inv"] = df.at[i, "Int by Mort Hold"] * (passthrough_rate / mortgage_rate)
            df.at[i, "Scheduled Princ."] = df.at[i, "Month. Paym."] - df.at[i, "Int by Mort Hold"]
            df.at[i, "Prepaym."] = (df.at[i, "Start Month. Bal."]- df.at[i, "Scheduled Princ."])*(1-(1-df.at[i, 'CPR'])**(1/12))
            df.at[i, "Total Princ."] = df.at[i, "Scheduled Princ."] + df.at[i, "Prepaym."]
            df.at[i, "End Month. Bal."] = df.at[i, "Start Month. Bal."] - df.at[i, "Total Princ."]
    #cols_to_format = [col for col in df.columns if col not in ['CPR', 'SMM']]
    #df[cols_to_format] = df[cols_to_format].applymap(lambda x: f"{float(x):.2f}" if x not in [None, 'None'] else x)
    
    print("Sum of monthly payments:", df["Month. Paym."].apply(pd.to_numeric, errors='coerce').sum())
    print("Sum of interest paid by mortgage holders:", df["Int by Mort Hold"].apply(pd.to_numeric, errors='coerce').sum())
    print("Sum of interest paid to pass-through investor:", df["Int to P-T Inv"].apply(pd.to_numeric, errors='coerce').sum())
    print("Interest earned by MBS issuer:", df["Int by Mort Hold"].apply(pd.to_numeric, errors='coerce').sum() - df["Int to P-T Inv"].apply(pd.to_numeric, errors='coerce').sum())
    print("Sum of prepayments:", df["Prepaym."].apply(pd.to_numeric, errors='coerce').sum())
    print("Sum of total principal payments:", df["Total Princ."].apply(pd.to_numeric, errors='coerce').sum())

    df["Discount Rate"] = None
    df["Interest Rate"] = interest_rate
    for i in df.index:
        if df.at[i, "End Month. Bal."] is not None:
            df.at[i, "Discount Rate"] = 1/(1+interest_rate / 12)**i
    df["Principal Only (PO) PV"] = None
    for i in df.index:
        if df.at[i, "End Month. Bal."] is not None:
            df.at[i, "Principal Only (PO) PV"] = df.at[i, "Total Princ."] * df.at[i, "Discount Rate"]
    df["Interest Only (IO) PV"] = None
    for i in df.index: 
        if df.at[i, "End Month. Bal."] is not None:
            df.at[i, "Interest Only (IO) PV"] = df.at[i, "Int to P-T Inv"] * df.at[i, "Discount Rate"]

    print("Present Value of Principal Only (PO) tranche:", df["Principal Only (PO) PV"].apply(pd.to_numeric, errors='coerce').sum())
    print("Present Value of Interest Only (IO) tranche:", df["Interest Only (IO) PV"].apply(pd.to_numeric, errors='coerce').sum())  
    
    return df

In [66]:
df = cpr_smm_dataframe(400000, 0.06,0.05, 240, 0.035, prepayment_multiplier=1.5, seasoning=2)
df

Sum of monthly payments: 341147.0254882459
Sum of interest paid by mortgage holders: 174326.24630917484
Sum of interest paid to pass-through investor: 145271.87192431238
Interest earned by MBS issuer: 29054.374384862458
Sum of prepayments: 233179.22082092907
Sum of total principal payments: 400000.0000000001
Present Value of Principal Only (PO) tranche: 314840.4133967396
Present Value of Interest Only (IO) tranche: 121656.55229037162


,CPR,SMM,Start Month. Bal.,Month. Paym.,Int by Mort Hold,Int to P-T Inv,Scheduled Princ.,Prepaym.,Total Princ.,End Month. Bal.,Discount Rate,Interest Rate,Principal Only (PO) PV,Interest Only (IO) PV
1,0.009,0.000753,400000,2878.214079,2000.0,1666.666667,878.214079,300.583269,1178.797348,398821.202652,0.997092,0.035,1175.369188,1661.819693
2,0.012,0.001006,398821.202652,2876.046462,1994.106013,1661.755011,881.940449,400.144856,1282.085305,397539.117346,0.994192,0.035,1274.639068,1652.103685
3,0.015,0.001259,397539.117346,2873.154475,1987.695587,1656.412989,885.458889,499.258844,1384.717733,396154.399613,0.991301,0.035,1372.671789,1642.003512
4,0.018,0.001513,396154.399613,2869.538102,1980.771998,1650.643332,888.766104,597.846863,1486.612966,394667.786647,0.988418,0.035,1469.39488,1631.525431
5,0.021,0.001767,394667.786647,2865.19787,1973.338933,1644.449111,891.858937,695.83091,1587.689848,393080.096799,0.985543,0.035,1564.737262,1620.67598
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
236,0.090,0.007828,1491.438178,502.125785,7.457191,6.214326,494.668594,7.803131,502.471725,988.966452,0.502916,0.035,252.701237,3.125286
237,0.090,0.007828,988.966452,498.194933,4.944832,4.120694,493.250101,3.880676,497.130777,491.835675,0.501454,0.035,249.288096,2.066337
238,0.090,0.007828,491.835675,494.294854,2.459178,2.049315,491.835675,-0.0,491.835675,-0.0,0.499995,0.035,245.915594,1.024648
239,0.090,0.007828,None,None,None,None,None,None,None,None,None,0.035,None,None


In [27]:
import random

def random_tranche_split(mortgage_balance):
    """
    Randomly selects a number n from 3 to 9, then splits the mortgage_balance into n decreasing parts.
    Assigns each part a coupon rate from (2,9), strictly increasing from lowest to highest.

    Returns:
        List of dicts: [{'tranche': i, 'balance': ..., 'coupon_rate': ...}, ...]
    """
    n = random.randint(3, 9)
    # Generate n-1 random breakpoints in (0, 1) to split proportionally
    cuts = sorted([random.uniform(0, 1) for _ in range(n-1)], reverse=True)
    proportions = [cuts[0]] + [cuts[i] - cuts[i+1] for i in range(n-2)] + [1 - cuts[-1]]
    # Normalize to ensure sum exactly 1 (floating point safety)
    proportions = [p / sum(proportions) for p in proportions]
    balances = [mortgage_balance * p for p in proportions]
    # Ensure strictly decreasing order
    balances = sorted(balances, reverse=True)
    # Generate n coupon rates in (2,9), strictly increasing
    coupon_rates = sorted([random.uniform(2, 9)/100 for _ in range(n)])
    tranches = []
    for i in range(n):
        tranches.append({
            'tranche': i+1,
            'balance': balances[i],
            'coupon_rate': coupon_rates[i]
        })
    return tranches

# Example usage:
# tranches = random_tranche_split(400000)
# print(tranches)


In [31]:
random_split = random_tranche_split(400000)

In [32]:
sum_balances = sum(tranche['balance'] for tranche in random_split)
print("Sum of balances in random_split:", sum_balances)

Sum of balances in random_split: 400000.0


In [72]:
def sequential_pay_CMO(df):
    """
    Create a sequential pay CMO structure from the given mortgage cash flow DataFrame.

    Args:
        df (pd.DataFrame): DataFrame contain-ing mortgage cash flows.
    """

    valid_indices = df[df["End Month. Bal."].notna()].index
    cmo_df = pd.DataFrame(index=valid_indices)
    tranche_split = random_tranche_split(df.at[1, "Start Month. Bal."])
    #tranche_split = [
    #    {'tranche': 1, 'balance': 150000, 'coupon_rate': 0.05},
    #    {'tranche': 2, 'balance': 100000, 'coupon_rate': 0.05},
    #    {'tranche': 3, 'balance': 90000, 'coupon_rate': 0.05},
    #    {'tranche': 4, 'balance': 60000, 'coupon_rate': 0.05},
    #]
    for idx, tranche in enumerate(tranche_split, 1):
        cmo_df[f"t{idx}_Bal"] = None
        cmo_df[f"t{idx}_princ"] = None
        cmo_df[f"t{idx}_int"] = None
    
    # Add MultiIndex columns to group each tranche's columns under "Tranche 1", "Tranche 2", etc.
    tranche_names = [f"Tranche {i+1}" for i in range(len(tranche_split))]
    columns = []
    for name, idx in zip(tranche_names, range(1, len(tranche_split)+1)):
        columns.extend([
            (name, f"t{idx}_Bal"),
            (name, f"t{idx}_princ"),
            (name, f"t{idx}_int"),
        ])
    cmo_df.columns = pd.MultiIndex.from_tuples(columns)
    
    # Set initial balances for each tranche in the first row
    for idx, tranche in enumerate(tranche_split, 1):
        cmo_df.at[valid_indices[0], (f"Tranche {idx}", f"t{idx}_Bal")] = tranche['balance']
   
    # Fill principal payments for the first row (sequential pay)
    total_princ = float(df.at[valid_indices[0], "Total Princ."])
    remaining_princ = total_princ
    for idx, tranche in enumerate(tranche_split, 1):
        tranche_bal = float(tranche['balance'])
        paid = min(remaining_princ, tranche_bal)
        cmo_df.at[valid_indices[0], (f"Tranche {idx}", f"t{idx}_princ")] = paid
        remaining_princ -= paid
        if remaining_princ < 0:
            remaining_princ = 0
    

    # Fill interest payments for the first row
    for idx, tranche in enumerate(tranche_split, 1):
        tranche_bal = float(tranche['balance'])
        tranche_coupon = float(tranche['coupon_rate'])
        interest = max(0, tranche_bal * tranche_coupon / 12)
        cmo_df.at[valid_indices[0], (f"Tranche {idx}", f"t{idx}_int")] = interest

    # Fill remaining rows for balances, principal, and interest
    for row_idx in valid_indices[1:]:
        prev_idx = row_idx - 1

        # Track remaining principal to allocate for this period
        total_princ = float(df.at[row_idx, "Total Princ."])
        remaining_princ = total_princ

        for idx, tranche in enumerate(tranche_split, 1):
            bal_col = (f"Tranche {idx}", f"t{idx}_Bal")
            princ_col = (f"Tranche {idx}", f"t{idx}_princ")
            int_col = (f"Tranche {idx}", f"t{idx}_int")

            # Previous balance
            prev_bal = cmo_df.at[prev_idx, bal_col]
            prev_bal = float(prev_bal) if prev_bal not in [None, ""] else 0

            # Calculate new balance
            prev_princ = cmo_df.at[prev_idx, princ_col]
            prev_princ = float(prev_princ) if prev_princ not in [None, ""] else 0

            if prev_bal in [None, ""] or prev_bal <= 0:
                cmo_df.at[row_idx, bal_col] = ""
                cmo_df.at[row_idx, princ_col] = 0
                cmo_df.at[row_idx, int_col] = 0
            else:
                new_bal = prev_bal - prev_princ
                cmo_df.at[row_idx, bal_col] = new_bal

                # Principal payment for this tranche
                if new_bal is None or new_bal <= 0:
                    princ_pay = 0
                else:
                    princ_pay = min(remaining_princ, new_bal)
                cmo_df.at[row_idx, princ_col] = princ_pay
                remaining_princ -= princ_pay
                if remaining_princ < 0:
                    remaining_princ = 0

                # Interest payment for this tranche
                coupon = float(tranche['coupon_rate'])
                interest = max(0, new_bal * coupon / 12)
                cmo_df.at[row_idx, int_col] = interest

    # Extend filling for remaining tranches (t2, t3, ...)
    for row_idx in valid_indices[1:]:
        prev_idx = row_idx - 1
        total_princ = float(df.at[row_idx, "Total Princ."])
        # For each tranche, update balance and principal sequentially
        for idx in range(2, len(tranche_split)+1):
            bal_col = (f"Tranche {idx}", f"t{idx}_Bal")
            princ_col = (f"Tranche {idx}", f"t{idx}_princ")
            int_col = (f"Tranche {idx}", f"t{idx}_int")

            prev_bal = cmo_df.at[prev_idx, bal_col]
            prev_bal = float(prev_bal) if prev_bal not in [None, "", ""] else 0

            prev_princ = cmo_df.at[prev_idx, princ_col]
            prev_princ = float(prev_princ) if prev_princ not in [None, "", ""] else 0

            # Update balance
            if prev_bal in [None, "", ""] or prev_bal <= 0:
                cmo_df.at[row_idx, bal_col] = ""
            else:
                cmo_df.at[row_idx, bal_col] = prev_bal - prev_princ

            # Calculate principal payment for this tranche
            # Deduct all previous tranches' principal from total_princ
            prev_tranches_princ = sum(
                float(cmo_df.at[row_idx, (f"Tranche {j}", f"t{j}_princ")]) if cmo_df.at[row_idx, (f"Tranche {j}", f"t{j}_princ")] not in [None, "", ""] else 0
                for j in range(1, idx)
            )
            tranche_bal = cmo_df.at[row_idx, bal_col]
            tranche_bal = float(tranche_bal) if tranche_bal not in [None, "", ""] else 0

            if tranche_bal in [None, "", ""] or tranche_bal <= 0:
                cmo_df.at[row_idx, princ_col] = 0
            else:
                avail_princ = total_princ - prev_tranches_princ
                avail_princ = max(avail_princ, 0)
                princ_pay = min(avail_princ, tranche_bal)
                cmo_df.at[row_idx, princ_col] = princ_pay

            # Interest payment for this tranche
            coupon = float(tranche_split[idx-1]['coupon_rate'])
            interest = max(0, tranche_bal * coupon / 12)
            cmo_df.at[row_idx, int_col] = interest

    return cmo_df

In [73]:
sequential_pay_CMO(df)

Tranche 1                               Tranche 2           \
            t1_Bal     t1_princ      t1_int         t2_Bal t2_princ   
1    135791.905199  1178.797348  557.145042  134069.923764      0.0   
2    134613.107851  1282.085305  552.308516  134069.923764      0.0   
3    133331.022546  1384.717733  547.048206  134069.923764      0.0   
4    131946.304813  1486.612966  541.366803  134069.923764      0.0   
5    130459.691846  1587.689848   535.26733  134069.923764      0.0   
..             ...          ...         ...            ...      ...   
234                           0           0                       0   
235                           0           0                       0   
236                           0           0                       0   
237                           0           0                       0   
238                           0           0                       0   

                    Tranche 3                          Tranche 4  ...  \
         t2_int        t3_Bal t3_princ      t3_int        t4_Bal  ...   
1    562.447449  67367.250316      0.0  313.374799  22510.384795  ...   
2    562.447449  67367.250316      0.0  313.374799  22510.384795  ...   
3    562.447449  67367.250316      0.0  313.374799  22510.384795  ...   
4    562.447449  67367.250316      0.0  313.374799  22510.384795  ...   
5    562.447449  67367.250316      0.0  313.374799  22510.384795  ...   
..          ...           ...      ...         ...           ...  ...   
234           0                      0           0                ...   
235           0                      0           0                ...   
236           0                      0           0                ...   
237           0                      0           0                ...   
238           0                      0           0                ...   

                    Tranche 5                         Tranche 6           \
         t4_int        t5_Bal t5_princ     t5_int        t6_Bal t6_princ   
1    137.949353  14650.153321      0.0  94.695652  13908.060423      0.0   
2    137.949353  14650.153321      0.0  94.695652  13908.060423      0.0   
3    137.949353  14650.153321      0.0  94.695652  13908.060423      0.0   
4    137.949353  14650.153321      0.0  94.695652  13908.060423      0.0   
5    137.949353  14650.153321      0.0  94.695652  13908.060423      0.0   
..          ...           ...      ...        ...           ...      ...   
234           0                      0          0                      0   
235           0                      0          0                      0   
236           0                      0          0                      0   
237           0                      0          0                      0   
238           0                      0          0                      0   

                   Tranche 7                         
        t6_int        t7_Bal    t7_princ     t7_int  
1    94.237634  11702.322181         0.0  82.656473  
2    94.237634  11702.322181         0.0  82.656473  
3    94.237634  11702.322181         0.0  82.656473  
4    94.237634  11702.322181         0.0  82.656473  
5    94.237634  11702.322181         0.0  82.656473  
..         ...           ...         ...        ...  
234          0   2512.589732   513.29266  17.747059  
235          0   1999.297072  507.858894  14.121543  
236          0   1491.438178  502.471725  10.534407  
237          0    988.966452  497.130777   6.985321  
238          0    491.835675  491.835675    3.47396  

[238 rows x 21 columns]